# Deploy to BYOC on AKS

description: deploy a BYOC container on AKS

In [2]:
from azureml.core import Workspace

ws = Workspace.from_config()
ws

Workspace.create(name='Inference-PM-AML-Workspace', subscription_id='92c76a2f-0e1c-4216-b65e-abf7a3f34c1e', resource_group='Inference-PM')

In [8]:
!docker build --tag inferencepmaef584480.azurecr.io/flaskproject:no-entry-chmod .
!docker push inferencepmaef584480.azurecr.io/flaskproject:no-entry-chmod

[+] Building 0.0s (0/1)                                                         
[+] Building 0.1s (2/2)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 38B                                        0.0s
 => [internal] load .dockerignore                                          0.1s
 => => transferring context: 2B                                            0.0s
[+] Building 0.3s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 38B                                        0.0s
 => [internal] load .dockerignore                                          0.1s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/python:3                0.1s
[+] Building 0.4s (2/3)              

In [6]:
from azureml.core.compute import AksCompute
from azureml.exceptions import ComputeTargetException

prov_config = AksCompute.provisioning_configuration()

try:
    test = AksCompute.create(workspace=ws, name='gopalv-byoc-test', provisioning_configuration=prov_config)
except ComputeTargetException:
    test = AksCompute(ws, 'gopalv-byoc-test')

test.wait_for_completion(show_output=True)

ComputeTargetException: ComputeTargetException:
	Message: Long running operation information not known, unable to poll. Current state is Succeeded
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Long running operation information not known, unable to poll. Current state is Succeeded"
    }
}

## Deploy webservice

In this case we deploy to the local compute, but for other options, see [our documentation](https://docs.microsoft.com/azure/machine-learning/how-to-deploy-and-where?tabs=azcli).


In [14]:
from azureml.core.webservice import AksWebservice
from azureml.core import Environment
from azureml.core.model import Model
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from random import randint
from azureml.core.model import Model

service_name = "gopalv-byoc-test-1"


env = Environment("byoc-test")

env.docker.base_image = "inferencepmaef584480.azurecr.io/flaskproject:no-entry-chmod"

# Need these to tell AML not to install our default components
env.python.user_managed_dependencies = True
env.inferencing_stack_version = None

env.environment_variables["WORKER_COUNT"] = "1"

inference_config = InferenceConfig(
    # this entry script is where we dispatch a call to the Triton server
    source_directory="src",
    entry_script="app.py",
    environment=env,
)

config = AksWebservice.deploy_configuration(
    compute_target_name="gopalv-byoc-test",
)

service = Model.deploy(
    workspace=ws,
    name=service_name,
    models=[],
    inference_config=inference_config,
    deployment_config=config
)

service.wait_for_deployment(show_output=True)

WARNING - Warning, custom base image or base dockerfile detected without a specified `inferencing_stack_version`. Please set environment.inferencing_stack_version='latest'
Running.........
Succeeded
AKS service creation operation finished, operation "Succeeded"


In [ ]:
print(service.get_logs())

In [24]:
base_uri = service.scoring_uri[:-5]
token = service.get_keys()[0]

In [31]:
import requests

result = requests.get(url=base_uri, data={"test": 0}, headers={"Authorization": f"Bearer {token}"})
print(result.content)

b'Hello, world!'


In [10]:
print(ws.webservices)

{'triton-densenet-onnx-local79907': AksWebservice(workspace=Workspace.create(name='Inference-PM-AML-Workspace', subscription_id='92c76a2f-0e1c-4216-b65e-abf7a3f34c1e', resource_group='Inference-PM'), name=triton-densenet-onnx-local79907, image_id=None, compute_type=None, state=AKS, scoring_uri=None, tags=http://52.166.71.20:80/api/v1/service/triton-densenet-onnx-local79907/score, properties={}, created_by={'azureml.git.repository_uri': 'git@github.com:Azure/azureml-examples.git', 'mlflow.source.git.repoURL': 'git@github.com:Azure/azureml-examples.git', 'azureml.git.branch': 'ey-hack', 'mlflow.source.git.branch': 'ey-hack', 'azureml.git.commit': 'ae4a3b0ac5a284c9b035b950c7bf01ca152af389', 'mlflow.source.git.commit': 'ae4a3b0ac5a284c9b035b950c7bf01ca152af389', 'azureml.git.dirty': 'True', 'hasInferenceSchema': 'False', 'hasHttps': 'False'}), 'triton-densenet-onnx-local75850': AksWebservice(workspace=Workspace.create(name='Inference-PM-AML-Workspace', subscription_id='92c76a2f-0e1c-4216-b

In [13]:
service = AksWebservice(ws, "gopalv-byoc-test-1")
service.delete()